In [11]:
import localmd
import matplotlib.pyplot as plt


import scipy
import scipy.sparse
import jax
import jax.scipy
import jax.numpy as jnp
from jax import jit, vmap
import numpy as np
import math

import os
import tifffile

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Specify the dataset in the below panel. See PMD documentation for easy ways to add support for your custom data formats. We provide default support for single, multipage tiff files. 

In [6]:
input_file = "../datasets/demoMovie.tif"
current_dataset = localmd.TiffArray(input_file)

# Specify params, run PMD

In [7]:
block_sizes = [32, 32]
frame_range = 5000
max_components = 20
background_rank = 1
rank_pruning = True

current_video = localmd.localmd_decomposition(current_dataset,
                                               block_sizes,
                                               frame_range,
                                               max_components=max_components, 
                                               background_rank=background_rank,
                                               sim_conf=5,
                                               frame_batch_size = 1000,
                                               dtype='float32',
                                               pixel_batch_size=5000,
                                               max_consecutive_failures = 1,
                                               rank_prune=rank_pruning,
                                               temporal_avg_factor=10)

# The PMDArray object is a convenient representation of the PMD decomposition of the data. You can use this object to interact with the decomposition via standard "array-like" functionality: 


### CAUTION: Doing something like current_video[:, :, :] will involve returning the full dataset. You'll notice PMD is a massively compressed representation of the data and can fit into your system's RAM, thanks to a matrix factorization. By running current_video[:, :, :] you are expanding out that matrix factorization and explicitly returning a matrix whose dimensions is the dimensions of your full movie: for big data this will construct a movie of shape (data_shape). Instead, intended use is to look at subsets of the data efficiently (load some frames, spatially crop the data, and do combinations of these two operations) like below: 

In [1]:
#Here is how to plot single pixels of the data:
plt.figure()
plt.plot(current_video[:, 30, 40])
plt.show()

#Here is how to work with frames of the data: 
plt.figure()
plt.imshow(current_video[100, :, :])
plt.show()


#Here is how to do both combinations of things: 
plt.figure()
plt.imshow(current_video[50, 20:40, 10:30])
plt.show()

# Save the individual matrices of the PMD decomposition into a NPZ file

In [13]:
npz_save_name = "INSERT_SAVE_NAME_HERE.npz"
U = current_video.U_sparse
R = current_video.R
s = current_video.s
V = current_video.V
mean_img = current_video.mean_img
std_img = current_video.var_img
data_shape = current_video.shape
order = current_video.order


np.savez(npz_save_name, fov_shape = data_shape[1:], \
                fov_order=order, U_data = U.data, \
                U_indices = U.indices,\
                U_indptr=U.indptr, \
                U_shape = U.shape, \
                U_format = type(U), \
                R = R, \
                s = s, \
                Vt = V, \
                mean_img = mean_img, \
                noise_var_img = std_img)

# How to load the data from our standard .npz representation

In [17]:
data = np.load(npz_save_name, allow_pickle=True)
U = scipy.sparse.csr_matrix(
    (data['U_data'], data['U_indices'], data['U_indptr']),
    shape=data['U_shape']
).tocoo()
V = data['Vt']
R = data['R']
s = data['s']
mean_img = data['mean_img']
std_img = data['noise_var_img']
data_shape = (V.shape[1], data['fov_shape'][0], data['fov_shape'][1])
data_order = data['fov_order'].item()

current_video = PMDArray(U, R, s, V, data_shape, data_order, mean_img, std_img)

# Generate a comparison triptych to show how well PMD retains signal from the original movie

In [18]:
cropped_dataset = current_dataset[:, :, :]
cropped_pmd = current_video[:, :, :]
cropped_residual = cropped_dataset - cropped_pmd

output_triptych = np.concatenate([cropped_dataset, cropped_pmd, cropped_residual], axis = 2)

## Modify the filename below as desired
filename_to_save = "Denoised_Vs_Raw_Comparison.tiff"

#The below line saves the tiff file
tifffile.imwrite(filename_to_save, output_triptych)